In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from shutil import copyfile

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator

/home/diegues/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def printWrongPreds(preds, targets):
    df = pd.DataFrame()
    images = []
    predictions = []
    targets_ = []
    if(type(preds) == pd.DataFrame):          
        predsClass = preds.idxmax(axis=1)
        targetsClass = targets.idxmax(axis=1)  
        for i in range(0,len(predsClass)):
            if(predsClass[i] != targetsClass[i]):
                images.append(targets.index[i])
                predictions.append(predsClass[i])
                targets_.append(targetsClass[i])
                print('Image:', targets.index[i],'\t\tPrediction:', predsClass[i], '\tTarget:', targetsClass[i])
    else:
        for i in range(0,len(preds)):
            if(preds[i] != targets['level3'][i]):
                images.append(targets.index[i])
                predictions.append(preds[i])
                targets_.append(targets['level3'][i])
                print('Image:', targets.index[i],'\t\tPrediction:', preds[i], '\tTarget:', targets['level3'][i])
    df['image'] = images
    df['prediction'] = predictions
    df['target'] = targets_
    return df

def createDirectoryEnvironment(train_data, train_targets, val_data, val_targets, path):
    trainpath = path + 'train/'
    valpath = path + 'validation/'
    if(os.listdir() == 2):
        return trainpath,valpath
    if(not os.path.exists(trainpath)):
        os.mkdir(trainpath)
    if(not os.path.exists(valpath)):
        os.mkdir(valpath)
        
    classes = train_targets.columns.append(val_targets.columns).drop_duplicates()  
    for c in classes:
        train_class = trainpath + c + '/'
        val_class = valpath + c + '/'
        if(not os.path.exists(train_class)):
            os.mkdir(train_class)
        if(not os.path.exists(val_class)):
            os.mkdir(val_class)
            
    for f in train_data.index.values:
        copyfile(path + '../' + f[:f.find('frame')-1] + '/' + f, trainpath + train_targets.loc[f].idxmax(axis = 1) + '/' + f)
    for f in val_data.index.values:
        copyfile(path + '../' + f[:f.find('frame')-1] + '/' + f, valpath + val_targets.loc[f].idxmax(axis = 1) + '/' + f)
    
    return trainpath, valpath

In [4]:
folder_path = "/home/diegues/Desktop/ProcessedImages/"
data = pd.read_csv(folder_path + "sampled_data.csv")
#classes = open(folder_path + "classes.txt", "r").readlines()

filenames = data['filename']
targets = data['level3']

# one-hot encoding
targets_ohe = pd.get_dummies(data['level3'])
#species_ohe = pd.get_dummies(data['species'])

# dealing with NaNs
data = data.drop(['roll', 'pitch', 'level1', 'level2', 'level3', 'level4', 
                  'level5', 'level6', 'AphiaID', 'EunisName', 'EunisCode', 
                  'date', 'timestamp', 'species'],
                 axis = 1)

X = data.groupby('filename').max()
Y_ohe = pd.concat([filenames,targets_ohe], axis = 1).groupby('filename').max()
Y_cat = pd.concat([filenames,targets], axis = 1).groupby('filename').max()

print(pd.value_counts(Y_cat.level3).to_frame().reset_index())

# tts
train_X_ohe, test_X_ohe, train_Y_ohe, test_Y_ohe = train_test_split(X, Y_ohe, test_size = 0.3)
train_X_cat, test_X_cat, train_Y_cat, test_Y_cat = train_test_split(X, Y_cat, test_size = 0.3)

  index  level3
0  A5.1     178
1  A4.1     178
2  A3.1     175
3  A4.7     174
4  A3.7      91
5  A5.2      85
6  A5.4      34


In [7]:
# Random Forest prediction
rf = RandomForestClassifier(n_estimators = 1000)
rf.fit(train_X_ohe, train_Y_ohe)
predictions_rf = rf.predict(test_X_ohe)
predictions_rf = pd.DataFrame(predictions_rf)
predictions_rf.columns = test_Y_ohe.columns.values
print('RF:\t',rf.score(test_X_ohe, test_Y_ohe))
failed_rf = printWrongPreds(predictions_rf, test_Y_ohe)
print(pd.DataFrame([(name,round(value,3)) for name,value in zip(X.columns,rf.feature_importances_)]))


RF:	 0.9490909090909091
Image: 132143_forcadinho-np3_frame2809.jpg 		Prediction: A3.1 	Target: A4.1
Image: 113610_cam_survey_1_frame380.jpg 		Prediction: A3.7 	Target: A3.1
Image: 105317_cam-np3_frame354.jpg 		Prediction: A3.1 	Target: A4.1
Image: 132143_forcadinho-np3_frame4304.jpg 		Prediction: A3.1 	Target: A4.1
Image: 132143_forcadinho-np3_frame1129.jpg 		Prediction: A3.1 	Target: A5.1
Image: 105317_cam-np3_frame481.jpg 		Prediction: A3.1 	Target: A4.1
Image: 105317_cam-np3_frame304.jpg 		Prediction: A4.7 	Target: A4.1
Image: 105317_cam-np3_frame260.jpg 		Prediction: A4.1 	Target: A4.7
Image: 104728_cam-np3_frame62.jpg 		Prediction: A4.7 	Target: A4.1
Image: 132143_forcadinho-np3_frame2784.jpg 		Prediction: A3.1 	Target: A4.1
Image: 132143_forcadinho-np3_frame4062.jpg 		Prediction: A3.1 	Target: A3.7
Image: 125355_forcadinho-np3_frame273.jpg 		Prediction: A5.1 	Target: A4.1
Image: 132143_forcadinho-np3_frame1394.jpg 		Prediction: A3.1 	Target: A5.1
Image: 104728_cam-np3_frame68.jpg

In [8]:
# Support Vector Machines
svm = svm.SVC()
svm.fit(train_X_cat, train_Y_cat.values.ravel())
preds_svm = svm.predict(test_X_cat)
print('SVM:\t',accuracy_score(test_Y_cat, preds_svm))
#failed_svm = printWrongPreds(preds_svm, test_Y_cat) # too many wrong predictions to print it

SVM:	 0.6290909090909091


In [9]:
# Neural Networks
## sklearn

scaler = StandardScaler()
scaler.fit(train_X_cat)

train_X_scaled = scaler.transform(train_X_cat)
test_X_scaled = scaler.transform(test_X_cat)

mlp = MLPClassifier(hidden_layer_sizes=(4096,4096,1000))
mlp.fit(train_X_scaled,train_Y_cat.values.ravel())

predictions_nn = mlp.predict(test_X_scaled)
print('sklearn NN:\t',accuracy_score(test_Y_cat,predictions_nn))
failed_nn = printWrongPreds(predictions_nn, test_Y_cat)


sklearn NN:	 0.8981818181818182
Image: 132143_forcadinho-np3_frame4215.jpg 		Prediction: A3.1 	Target: A3.7
Image: 105317_cam-np3_frame979.jpg 		Prediction: A4.7 	Target: A4.1
Image: 105317_cam-np3_frame881.jpg 		Prediction: A4.1 	Target: A5.2
Image: 105317_cam-np3_frame789.jpg 		Prediction: A5.2 	Target: A4.1
Image: 125355_forcadinho-np3_frame1813.jpg 		Prediction: A5.1 	Target: A3.1
Image: 132143_forcadinho-np3_frame3012.jpg 		Prediction: A3.1 	Target: A3.7
Image: 132143_forcadinho-np3_frame1394.jpg 		Prediction: A3.1 	Target: A5.1
Image: 132143_forcadinho-np3_frame4238.jpg 		Prediction: A3.1 	Target: A3.7
Image: 132143_forcadinho-np3_frame1598.jpg 		Prediction: A3.1 	Target: A3.7
Image: 105317_cam-np3_frame489.jpg 		Prediction: A4.7 	Target: A4.1
Image: 113610_cam_survey_1_frame67.jpg 		Prediction: A3.7 	Target: A3.1
Image: 132143_forcadinho-np3_frame2969.jpg 		Prediction: A4.1 	Target: A3.7
Image: 113610_cam_survey_1_frame360.jpg 		Prediction: A3.7 	Target: A3.1
Image: 113610_cam_s

In [10]:
## keras

scaler_keras = StandardScaler()
scaler_keras.fit(train_X_ohe)

X_train_scaled = scaler.transform(train_X_ohe)
X_test_scaled = scaler.transform(test_X_ohe)

model = Sequential()
model.add(Dense(4096, activation='relu', input_dim=4))
model.add(Dropout(0.2))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(7, activation='sigmoid')) 

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_X_scaled, train_Y_ohe, epochs=20,validation_data=(test_X_scaled,test_Y_ohe))
predskeras_nn = pd.DataFrame(model.predict(test_X_scaled))
predskeras_nn.columns = test_Y_ohe.columns.values
score = model.evaluate(test_X_scaled, test_Y_ohe)

print('Keras NN:\t', score)
failed_keras = printWrongPreds(predskeras_nn,test_Y_ohe)

Train on 640 samples, validate on 275 samples
Epoch 1/50
640/640 [==============================] - 8s 12ms/step - loss: 0.4954 - acc: 0.8185 - val_loss: 0.4131 - val_acc: 0.8571
Epoch 2/50
640/640 [==============================] - 7s 11ms/step - loss: 0.4352 - acc: 0.8484 - val_loss: 0.4127 - val_acc: 0.8571
Epoch 3/50
640/640 [==============================] - 7s 11ms/step - loss: 0.4208 - acc: 0.8554 - val_loss: 0.4073 - val_acc: 0.8571
Epoch 4/50
640/640 [==============================] - 7s 11ms/step - loss: 0.4178 - acc: 0.8574 - val_loss: 0.4035 - val_acc: 0.8571
Epoch 5/50
640/640 [==============================] - 7s 11ms/step - loss: 0.4062 - acc: 0.8565 - val_loss: 0.4008 - val_acc: 0.8571
Epoch 6/50
640/640 [==============================] - 7s 11ms/step - loss: 0.4042 - acc: 0.8569 - val_loss: 0.3962 - val_acc: 0.8571
Epoch 7/50
640/640 [==============================] - 7s 11ms/step - loss: 0.4054 - acc: 0.8563 - val_loss: 0.3988 - val_acc: 0.8571
Epoch 8/50
640/640 [===

Image: 132143_forcadinho-np3_frame1751.jpg 		Prediction: A4.1 	Target: A5.1
Image: 105317_cam-np3_frame475.jpg 		Prediction: A5.1 	Target: A4.7
Image: 105317_cam-np3_frame404.jpg 		Prediction: A5.1 	Target: A4.1
Image: 105317_cam-np3_frame106.jpg 		Prediction: A5.1 	Target: A4.7
Image: 125355_forcadinho-np3_frame1230.jpg 		Prediction: A5.1 	Target: A3.1
Image: 113610_cam_survey_1_frame172.jpg 		Prediction: A5.1 	Target: A3.7
Image: 132143_forcadinho-np3_frame1685.jpg 		Prediction: A4.1 	Target: A5.4
Image: 125355_forcadinho-np3_frame1630.jpg 		Prediction: A4.7 	Target: A4.1
Image: 105317_cam-np3_frame629.jpg 		Prediction: A3.1 	Target: A4.1
Image: 105317_cam-np3_frame1035.jpg 		Prediction: A5.1 	Target: A4.1
Image: 113610_cam_survey_1_frame783.jpg 		Prediction: A4.1 	Target: A5.1
Image: 132143_forcadinho-np3_frame4216.jpg 		Prediction: A3.1 	Target: A3.7
Image: 113610_cam_survey_1_frame2013.jpg 		Prediction: A5.1 	Target: A3.1
Image: 132143_forcadinho-np3_frame2961.jpg 		Prediction: A3

In [22]:
# Image Classification

## Support Vector Machines
"""
classes = np.sort(np.array(Y_cat['EunisCode'].unique()))
class_map = dict((k,v) for (k, v) in zip(classes, [np.float32(i) for i in range(0,len(classes))]))
path_to_imgs = '/home/diegues/Desktop/ProcessedImages/LabeledData/'

X_train = []
X_test = []
y_train = []
y_test = []
i=0
for file in os.listdir(path_to_imgs):
    i = i + 1
    print(i)
    img = cv2.resize(cv2.imread(path_to_imgs + file, 0),(200,150))
    xarray_img = np.squeeze(np.array(img).astype(np.float32))
    m, v = cv2.PCACompute(xarray_img, mean = None)
    array = np.array(v)
    flat_array = array.ravel()
    if file in train_X_cat.index:
        X_train.append(flat_array)
        y_train.append(int(class_map[train_Y_cat['EunisCode'].loc[file]]))
        
    elif file in test_X_cat.index:
        X_test.append(flat_array)
        y_test.append(int(class_map[test_Y_cat['EunisCode'].loc[file]]))

X_train = np.float32(X_train)
X_test = np.float32(X_test)
y_train = np.float32(y_train)
y_test = np.float32(y_test)

img_svm = cv2.ml.SVM_create()
img_svm.setKernel(cv2.ml.SVM_LINEAR)
img_svm.setType(cv2.ml.SVM_C_SVC)
img_svm.setC(2.67)
img_svm.setGamma(5.383)
img_svm.train(X_train, cv2.ml.ROW_SAMPLE, y_train)
        
result = img_svm.predict(X_test)[1]
"""

"\nclasses = np.sort(np.array(Y_cat['EunisCode'].unique()))\nclass_map = dict((k,v) for (k, v) in zip(classes, [np.float32(i) for i in range(0,len(classes))]))\npath_to_imgs = '/home/diegues/Desktop/ProcessedImages/LabeledData/'\n\nX_train = []\nX_test = []\ny_train = []\ny_test = []\ni=0\nfor file in os.listdir(path_to_imgs):\n    i = i + 1\n    print(i)\n    img = cv2.resize(cv2.imread(path_to_imgs + file, 0),(200,150))\n    xarray_img = np.squeeze(np.array(img).astype(np.float32))\n    m, v = cv2.PCACompute(xarray_img, mean = None)\n    array = np.array(v)\n    flat_array = array.ravel()\n    if file in train_X_cat.index:\n        X_train.append(flat_array)\n        y_train.append(int(class_map[train_Y_cat['EunisCode'].loc[file]]))\n        \n    elif file in test_X_cat.index:\n        X_test.append(flat_array)\n        y_test.append(int(class_map[test_Y_cat['EunisCode'].loc[file]]))\n\nX_train = np.float32(X_train)\nX_test = np.float32(X_test)\ny_train = np.float32(y_train)\ny_test

In [16]:
## Convolutional Neural Networks - VGG config D

# Preping the data
images_path = '/home/diegues/Desktop/ProcessedImages/SampledData/'
#train_dir, val_dir = createDirectoryEnvironment(train_X_ohe, train_Y_ohe, test_X_ohe, test_Y_ohe, images_path)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        images_path + 'train',  
        target_size=(224, 224), 
        batch_size=32,
        class_mode='categorical') 

validation_generator = test_datagen.flow_from_directory(
        images_path + 'validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 640 images belonging to 7 classes.
Found 275 images belonging to 7 classes.


In [17]:
# Fit the models


vgg16 = Sequential()

vgg16.add(Conv2D(64,(3,3),activation='relu', input_shape=(224,224,3), padding='same'))
vgg16.add(Conv2D(64,(3,3),activation='relu', padding='same'))
vgg16.add(MaxPooling2D((2,2), (2,2)))
vgg16.add(Dropout(.25))

vgg16.add(Conv2D(128,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(128,(3,3),activation='relu', padding='same'))
vgg16.add(MaxPooling2D((2,2), (2,2)))
vgg16.add(Dropout(.5))

vgg16.add(Conv2D(256,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(256,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(256,(3,3),activation='relu', padding='same'))
vgg16.add(MaxPooling2D((2,2), (2,2)))
vgg16.add(Dropout(.5))

vgg16.add(Conv2D(512,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(512,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(512,(3,3),activation='relu', padding='same'))
vgg16.add(MaxPooling2D((2,2), (2,2)))
vgg16.add(Dropout(.5))

vgg16.add(Conv2D(512,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(512,(3,3),activation='relu', padding='same'))
vgg16.add(Conv2D(512,(3,3),activation='relu', padding='same'))
vgg16.add(MaxPooling2D((2,2), (2,2)))
vgg16.add(Dropout(.5))

vgg16.add(Flatten())
vgg16.add(Dense(1024, activation='relu'))
vgg16.add(Dropout(0.25))
vgg16.add(Dense(1024, activation='relu'))
vgg16.add(Dropout(0.5))
vgg16.add(Dense(100, activation='relu'))
vgg16.add(Dropout(0.75))
vgg16.add(Dense(7, activation='sigmoid'))

vgg16.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 56, 56, 128)       0         
__________

In [18]:
vgg16.fit_generator(
        train_generator,
        steps_per_epoch=len(train_X_ohe) // 32,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=len(test_X_ohe) // 32)
vgg16.save_weights('vgg16.h5')

Epoch 1/50
 1/20 [>.............................] - ETA: 26:40 - loss: 0.6944 - acc: 0.4554

KeyboardInterrupt: 

In [19]:
images_path = '/home/diegues/Desktop/ProcessedImages/SampledData/'

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        images_path + 'train',  
        target_size=(512, 512), 
        batch_size=32,
        class_mode='categorical') 

validation_generator = test_datagen.flow_from_directory(
        images_path + 'validation',
        target_size=(512, 512),
        batch_size=32,
        class_mode='categorical')

Found 640 images belonging to 7 classes.
Found 275 images belonging to 7 classes.


In [20]:
# DeepSense AI NOAA competition approach

dsaiNOAA = Sequential()

dsaiNOAA.add(Conv2D(32,(3,3),activation='relu', input_shape=(512,512,3), padding='same'))
dsaiNOAA.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
dsaiNOAA.add(Dropout(.25))

dsaiNOAA.add(Conv2D(64,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
dsaiNOAA.add(Dropout(.25))

dsaiNOAA.add(Conv2D(64,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(Conv2D(128,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(Conv2D(128,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
dsaiNOAA.add(Dropout(.25))

dsaiNOAA.add(Conv2D(256,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(Conv2D(256,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
dsaiNOAA.add(Dropout(.25))

dsaiNOAA.add(Conv2D(256,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(Conv2D(256,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
dsaiNOAA.add(Dropout(.5))

dsaiNOAA.add(Conv2D(256,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(Conv2D(256,(3,3),activation='relu', padding='same'))
dsaiNOAA.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
dsaiNOAA.add(Dropout(.5))

dsaiNOAA.add(Flatten())
dsaiNOAA.add(Dense(256, activation='relu'))
dsaiNOAA.add(Dense(64, activation='relu'))
dsaiNOAA.add(Dense(7, activation='sigmoid'))

dsaiNOAA.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
dsaiNOAA.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_75 (Conv2D)           (None, 512, 512, 32)      896       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 255, 255, 64)      18496     
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 127, 127, 64)      36928     
__________

In [21]:
dsaiNOAA.fit_generator(
        train_generator,
        steps_per_epoch=len(train_X_ohe) // 32,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=len(test_X_ohe) // 32)
dsaiNOAA.save_weights('dsaiNOAA.h5')

Epoch 1/50
 1/20 [>.............................] - ETA: 27:48 - loss: 0.6945 - acc: 0.4866

KeyboardInterrupt: 